## NAS connection and data download - local env (Version 1.0)

In [1]:
# Import packages
import os
import shutil
from smb.SMBConnection import SMBConnection
import tempfile
import re

import matplotlib.pyplot as plt
import numpy as np

import bigfish
import bigfish.stack as stack

### 1.1 Connect to the NAS

In [2]:
#specify the location of your data on the NAS
input_directory = ' ' #/your/input/directory
local_download_directory = ' ' #change to where you want to download your temp files

# NAS credentials and connection information
nas_host = ' ' #your_nas_server_hostname_or_ip'
nas_user = ' ' #your_username
nas_password = ' ' #your_password
nas_share = ' ' #(specific for the Nishimura lab - edit accordingly)

In [3]:
experiment_ID = input_directory.split(os.path.sep)[-1]
local_download_directory = os.path.join(local_download_directory, experiment_ID) #change to where you want to download your temp files

# Connect to the NAS
conn = SMBConnection(nas_user, nas_password, 'python-client', nas_host, use_ntlm_v2=True)
conn.connect(nas_host, 445)

# Check if connection is successfully established
smb_directory_path = os.path.relpath(input_directory, '/Volumes/WormHole')
shared_files = conn.listPath(nas_share, smb_directory_path) # move into folder with raw data
if shared_files:
    print("Connection is established.")
else:
    print("Connection is not established.")
    
# #print the filenames, save in a list 
# filenames = [shared_file.filename for shared_file in shared_files if shared_file.filename.endswith(("_R3D.dv","_R3D_REF.dv"))]
# filenames = sorted(filenames)
# filenames

Connection is established.


### 1.2 Download and organize

#### 1.2.1 List

In [4]:
# #Iterate through the files in the shared directory
shared_files = conn.listPath(nas_share, smb_directory_path) # move into folder with raw data

dv_paths = []
for shared_file in shared_files:
    file_path = os.path.join(smb_directory_path, shared_file.filename) 
    if shared_file.filename.endswith("_R3D.dv") or shared_file.filename.endswith("_R3D_REF.dv"):
        # If it does, append it to the dv_paths list
        dv_paths.append(file_path)
#print(dv_paths)

    
#print the filenames (not the entire path), save in a list 
filenames = [shared_file.filename for shared_file in shared_files if shared_file.filename.endswith(("_R3D.dv","_R3D_REF.dv"))]

    
# #Iterate through the shared_files list and print filenames that end with "R3D.dv" or "R3D_REF.dv" only
dv_files = []
for shared_file in shared_files:
    filename = shared_file.filename
    # Check if the filename ends with ".dv" or "d3d.dv"
    if filename.endswith("_R3D.dv") or filename.endswith("_R3D_REF.dv"):
        # If it does, append it to the dv_files list
        dv_files.append("/" + filename)

print("Files to download:", len(filenames))
print("Files to download:", filenames)

Files to download: 4
Files to download: ['231107_LP306_L4440_01_R3D.dv', '231107_LP306_L4440_01_R3D_REF.dv', '231107_LP306_L4440_02_R3D.dv', '231107_LP306_L4440_02_R3D_REF.dv']


#### 1.2.2 Download

In [5]:
# # Download the data from the NAS to a temporary directory in my local environment for processing
input_directory = os.path.join(local_download_directory, 'input')

# # Create the local download directory if it doesn't exist
os.makedirs(input_directory, exist_ok=True)
# Iterate through the list of remote .dv file paths and download each one.
# Then download the .dv file from the NAS to the local directory
for remote_path in dv_paths:
    filename = os.path.basename(remote_path)
    local_path = os.path.join(input_directory, filename)
    with open(local_path, 'wb') as local_file:
        conn.retrieveFile(nas_share, remote_path, local_file)

print(f"{len(dv_paths)} files downloaded to:{input_directory}") 

4 files downloaded to:/Users/nalytorres/Documents/01_bigfish/02_data/231107_LP306_L4440/input


#### 1.2.3 Organize

In [6]:
# Create a list to store the files to be moved
files_to_move = []
image_numbers = []
corrupted_files = []
# Define a regular expression pattern to extract the numeric identifier
pattern = r'_([^_]+)_R3D(?:|_REF)\.dv$'

# Iterate through the files in the directory
for filename in os.listdir(input_directory):
    match = re.search(pattern, filename)
    if match:
        # Extract the numeric identifier from the matched pattern
        image_number = match.group(1)
        files_to_move.append((filename, image_number))   
        
# Iterate through the files to be moved
for filename, image_number in files_to_move:
    # Create a new folder named after the image number
    new_folder = os.path.join(input_directory, image_number)
    os.makedirs(new_folder, exist_ok=True)
    
    # Construct the source and destination file paths
    source_path = os.path.join(input_directory, filename)
    destination_path = os.path.join(new_folder, filename)
    
    # count if there are two files, and sort them in their new directories. Move the file to the new folder
    shutil.move(source_path, destination_path)
    #print(f"Moved '({filename})' to '{image_number}'")

print("All files organized in subdirectories in the 'input' directory.")

All files organized in subdirectories in the 'input' directory.
